In [1]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip

wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION='curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE'
wget -N

https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,159 kB]
Get:14 http://security

In [5]:
!pip install chromedriver-autoinstaller

In [12]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import time
import hashlib
import os
import json
import sys
import chromedriver_autoinstaller
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')


def init_driver():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chromedriver_autoinstaller.install()
    return webdriver.Chrome(options=chrome_options)

def scroll_to_end(driver):
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

def crawl(url, driver, output_file, depth=0, max_depth=3, visited_links=set(), title_hashes=set()):
    # check max depth
    if depth > max_depth:
        return

    try:
        driver.get(url)
    except Exception as e:
        print(f"Error loading {url}: {e}")
        return

    time.sleep(2)
    scroll_to_end(driver)

    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    # rm unnesserary tags
    for tag in soup(['img', 'video']):
        tag.decompose()
    # get title -> create hash to check overlap
    #content = ' '.join(soup.get_text(separator=" ", strip=True).split())
    # get title
    try:
        title = soup.title.get_text()
    except Exception as e:
        print(f'Can not find title {url}: {e}')
        return
    #hash title
    title_hash = hashlib.md5(title.encode('utf-8')).hexdigest()

    if title_hash in title_hashes:
        return

    title_hashes.add(title_hash)
    visited_links.add(url)

    # find , handle child urls
    for link in soup.find_all('a', href=True):
        new_url = link['href']

        # rm rss
        if 'rss' in new_url:
            continue

        if ('video' in new_url) or ('media' in new_url) or ('podcast' in new_url):
            continue
        if ('#' in new_url) or ('@' in new_url) or ('javascript' in new_url):
            continue

        # change absolute
        if new_url.startswith('/'):
            new_url = url.rstrip('/') + new_url

        # rm redundant in url
        if ".htm" in new_url and new_url.count(".htm") > 1:
            new_url = new_url.replace(".htm", "", new_url.count(".htm") - 1)

        # recursive handle child url
        if new_url not in visited_links and new_url.startswith(url):
            crawl(new_url, driver, output_file, depth + 1, max_depth, visited_links, title_hashes)

    # get description
    try:
        meta_description = soup.find('meta', attrs={'property': 'og:description'}).get('content')
    except Exception as e:
        print(f'Can not get description from {url}:{e}')
        meta_description =''

    #metadata
    time_tag = soup.find_all("meta", {"class": "cms-date"})
    datetime_crawled = time_tag[0]['content'] if time_tag else "Not found"
    keywords_tag = soup.find('meta', attrs={'name': 'keywords'})
    keywords = keywords_tag.get('content') if keywords_tag else "No keywords"
    metadata={"datetime_crawled": datetime_crawled, "keywords": keywords}

    # get content
    try:
        content_all = soup.find_all("div", {"class": "article__body cms-body"})
        content_all = content_all[0].find_all('p')

        for tag in content_all:
            for attribute in ["class", "id", "name",'script', 'style', 'meta', 'link', 'img', 'video', 'a']:
                del tag[attribute]
        content_final=""
        for content in content_all[0:-1]:
            content_final=content_final+content.text.strip()
    except:
        return

    data ={
            "#url":url,
            "title": title,
            "description": meta_description,
            "content":content_final,
            "metadata":metadata
    }
    with open(output_file, 'a+', encoding='utf-8', newline='') as f:
        f.write(f'{data},\n')


def run_crawl(website_url, output_file):
    # check file if exist, rm
    if os.path.exists(output_file):
        os.remove(output_file)
    with open(output_file, 'w+', encoding='utf-8') as file:
        file.write('{\n')
        file.close()
    driver = init_driver()
    try:
        crawl(website_url, driver, output_file)
    finally:
        driver.quit()

#data config
website_url = 'https://baophapluat.vn/'
# chrome_driver_path = r'C:\Users\atong\Documents\chromedriver-win64\chromedriver.exe'
output_file = 'content.json'

#process content file
with open(output_file, 'a+', encoding='utf-8') as file:
    file.write('{\n')
    file.close()
run_crawl(website_url, output_file)

with open(output_file, 'a+', encoding='utf-8') as file:
    file.write('\n}')
    file.close()

Error loading https://baophapluat.vn/bds/ubnd-huyen-dan-phuong-de-xuat-thanh-pho-ha-noi-cap-so-do-cho-nhung-truong-hop-vi-pham-post526265.html: Message: timeout: Timed out receiving message from renderer: 299.730
  (Session info: chrome=129.0.6668.89)
Stacktrace:
#0 0x5a6ce8ddb02a <unknown>
#1 0x5a6ce8ac15e0 <unknown>
#2 0x5a6ce8aa9733 <unknown>
#3 0x5a6ce8aa9422 <unknown>
#4 0x5a6ce8aa7493 <unknown>
#5 0x5a6ce8aa7c4f <unknown>
#6 0x5a6ce8ab7667 <unknown>
#7 0x5a6ce8acd9b4 <unknown>
#8 0x5a6ce8ad2dfb <unknown>
#9 0x5a6ce8aa835e <unknown>
#10 0x5a6ce8acd80f <unknown>
#11 0x5a6ce8b54fe2 <unknown>
#12 0x5a6ce8b358c3 <unknown>
#13 0x5a6ce8b036b3 <unknown>
#14 0x5a6ce8b0468e <unknown>
#15 0x5a6ce8da5a2b <unknown>
#16 0x5a6ce8da99b1 <unknown>
#17 0x5a6ce8d92225 <unknown>
#18 0x5a6ce8daa532 <unknown>
#19 0x5a6ce8d7738f <unknown>
#20 0x5a6ce8dc9f28 <unknown>
#21 0x5a6ce8dca0f3 <unknown>
#22 0x5a6ce8dd9e7c <unknown>
#23 0x7a64b520eac3 <unknown>



In [19]:
with open('content.json','r',encoding='utf-8') as file:
  lines=file.readlines()
  print(len(lines))
  for line in lines[1:-1]:
    print(line)

361
{'#url': 'https://baophapluat.vn/thoi-su-chinh-tri/quoc-phong-an-ninh/kiem-tra-cong-tac-giao-duc-quoc-phong-va-an-ninh-tai-bo-doi-bien-phong-tinh-an-giang-post526453.html', 'title': 'Kiểm tra công tác giáo dục Quốc phòng và An ninh tại Bộ đội Biên phòng tỉnh An Giang | Báo Pháp luật Việt Nam điện tử', 'description': '(PLVN) - Sáng\xa024/9, Đoàn công tác của Bộ Tư lệnh Quân khu 9 do Thiếu tướng Quách Văn Nhỏ, Phó Tư lệnh Quân khu 9 làm trưởng đoàn tiến hành kiểm tra công tác giáo dục Quốc phòng và An ninh năm 2024 tại Bộ Chỉ huy Bộ đội Biên phòng tỉnh An Giang.', 'content': 'Tại đây, Đoàn đã tiến hành kiểm tra các nội dung liên quan đến công tác giáo dục Quốc phòng và An ninh của đơn vị như: Kết quả triển khai thực hiện Luật Giáo dục Quốc phòng và An ninh, văn bản chỉ đạo của Chính phủ, các Bộ, ngành Trung ương và Quân khu; công tác đăng ký, quản lý, xây dựng kế hoạch tổ chức thực hiện và kết quả bồi dưỡng kiến thức Quốc phòng và An ninh cho các đối tượng; kết quả phổ biến kiến thức

In [17]:
from google.colab import files

In [20]:
files.download("content.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
!python webserver.py

 * Serving Flask app 'webserver'
 * Debug mode: on
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 494-445-831
 * Detected change in '/content/webserver.py', reloading
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 494-445-831
 * Detected change in '/content/webserver.py', reloading
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 494-445-831
 * Detected change in '/content/webserver.py', reloading
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 494-445-831
 * Detected change in '/content/webserver.py', reloading
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 494-445-831
 * Detected change in '/content/webserver.py', reloading
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 494-445-831
 * Detected change in '/content/webserver.py', reloading
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 494-445-831
